In [1]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import palantir
import scanpy as sc
import scvelo as scv
import cellrank as cr
import scvi
import phate
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output
scvi.settings.seed = 1995
from anndata import AnnData
import time

import pandas as pd

import os
import sys
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

plotly_reds = matplotlib_to_plotly(reds, 255)
plotly_reds[0] = [0.0, 'rgb(211, 211, 211)']

# Inline plotting
%matplotlib inline

project_directory = '/Cranio_Lab/Louk_Seton/mesenchyme_project_2023'
os.chdir(os.path.expanduser("~")+project_directory)

Global seed set to 1995


In [2]:
adata =  sc.read('anndata_objects/mesenchyme_interactive.h5ad')

In [20]:
list(adata.obs['annotation_paper_coarse'].dtype.categories)

['Chondrocytes',
 'FNP Early Mesenchyme',
 'LNP Mesenchyme',
 'MNP Mesenchyme',
 'MXP Early Mesenchyme',
 'Maxillary Mesenchyme',
 'Nasal Cavity Mesenchyme',
 'Neural Crest',
 'Nostrils',
 'Osteoblasts',
 'Osteomesenchyme',
 'Palatal Shelf',
 'Periocular Mesenchyme',
 'Philtrum',
 'Primary Palate',
 'Roof - Sim1',
 'Upper Lip',
 'VNO Mesenchyme',
 'Whisker Pad']

In [21]:
adata[adata.obs['stage']=='8']

View of AnnData object with n_obs × n_vars = 815 × 19928
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'E8_mesen_annotation_coarse', 'E8_mesen_annotation_fine', 'integration_label', 'E9_mesen_annotation_coarse', 'E9_mesen_annotation_fine', 'E10_mesen_annotation_fine', 'E10_mesen_annotation_coarse', 'E11_mesen_annotation_coarse', 'E11_mesen_annotation_fine', 'E12_mesen_annotation_coarse', 'E13_mesen_annotation_coarse', 'E14_mesen_annotation_coarse', 'topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic

In [22]:
##E8
ecto = sc.read('anndata_objects/adata_ecto_E8_no_hesx1_pitx2.h5ad')
# mesen = sc.read('anndata_objects/adata_E8_mesen_annotated.h5ad')
mesen = adata[adata.obs['stage']=='8'].copy()

In [23]:
mesen

AnnData object with n_obs × n_vars = 815 × 19928
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'E8_mesen_annotation_coarse', 'E8_mesen_annotation_fine', 'integration_label', 'E9_mesen_annotation_coarse', 'E9_mesen_annotation_fine', 'E10_mesen_annotation_fine', 'E10_mesen_annotation_coarse', 'E11_mesen_annotation_coarse', 'E11_mesen_annotation_fine', 'E12_mesen_annotation_coarse', 'E13_mesen_annotation_coarse', 'E14_mesen_annotation_coarse', 'topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 't

In [24]:
###Ecto
ecto.obs['annotation_combined'] = ecto.obs['E8_ecto_annotation_coarse']
ecto.obs['annotation_combined_fine'] = ecto.obs['E8_ecto_annotation_coarse']


##add prefix
ecto.obs['annotation_combined'] = 'Ectoderm - ' + ecto.obs['annotation_combined'].astype(str)
ecto.obs['annotation_combined'] = ecto.obs['annotation_combined'].astype('category')

ecto.obs['annotation_combined_fine'] = 'Ectoderm - ' + ecto.obs['annotation_combined_fine'].astype(str)
ecto.obs['annotation_combined_fine'] = ecto.obs['annotation_combined_fine'].astype('category')

ecto.obs['annotation_combined_coarse'] = ecto.obs['annotation_combined']

###Mesen
mesen.obs['annotation_combined'] = mesen.obs['annotation_2024']
mesen.obs['annotation_combined_fine'] = mesen.obs['E8_mesen_annotation_fine']
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_paper_coarse']


##add prefix
mesen.obs['annotation_combined'] = 'Mesenchyme - ' + mesen.obs['annotation_combined'].astype(str)
mesen.obs['annotation_combined'] = mesen.obs['annotation_combined'].astype('category')

mesen.obs['annotation_combined_fine'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_fine'].astype(str)
mesen.obs['annotation_combined_fine'] = mesen.obs['annotation_combined_fine'].astype('category')

mesen.obs['annotation_combined_coarse'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_coarse'].astype(str)
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_combined_coarse'].astype('category')

In [25]:
ecto.obs['source']='ecto'
mesen.obs['source']='mesen'

In [26]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [27]:
adata_combined.obs['annotation_combined_coarse'].value_counts()

annotation_combined_coarse
Mesenchyme - Neural Crest            414
Mesenchyme - MXP Early Mesenchyme    251
Mesenchyme - FNP Early Mesenchyme    150
Ectoderm - Surface Ectoderm           77
Ectoderm - Ectodermal Placode         39
Name: count, dtype: int64

In [28]:
adata_combined.write('anndata_objects/adata_E8_combined_annotated.h5ad')

In [29]:
#E9
ecto = sc.read('anndata_objects/adata_ecto_E9_no_hesx1.h5ad')
#mesen = sc.read('anndata_objects/adata_E9_mesen_annotated.h5ad')
mesen = adata[adata.obs['stage']=='9'].copy()

In [31]:
ecto

AnnData object with n_obs × n_vars = 126 × 17204
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'leiden', 'leiden_ecto', 'E8_ecto_annotation_coarse', 'leiden_default', 'E9_ecto_annotation_coarse', 'reso_for_ecto', 'leiden_ecto_tmp', 'E10_ecto_annotation_coarse', 'leiden_ecto_fine', 'leiden_ecto_extra_fine', 'leiden_ecto1', 'E11_ecto_annotation_coarse', 'E11_ecto_annotation_fine', 'leiden_neuron', 'E12_ecto_annotation_coarse', 'E12_ecto_annotation_fine', 'leiden_coarse', 'E13_ecto_annotation_coarse', 'E13_ecto_annotation_fine', 'E14_ecto_annotation_coarse', 'E14_ecto_annotation_fine', 'annotation_tes

In [32]:
###Ecto
ecto.obs['annotation_combined'] = ecto.obs['E9_ecto_annotation_coarse']
ecto.obs['annotation_combined_fine'] = ecto.obs['E9_ecto_annotation_coarse']

##add prefix
ecto.obs['annotation_combined'] = 'Ectoderm - ' + ecto.obs['annotation_combined'].astype(str)
ecto.obs['annotation_combined'] = ecto.obs['annotation_combined'].astype('category')

ecto.obs['annotation_combined_fine'] = 'Ectoderm - ' + ecto.obs['annotation_combined_fine'].astype(str)
ecto.obs['annotation_combined_fine'] = ecto.obs['annotation_combined_fine'].astype('category')
ecto.obs['annotation_combined_coarse'] = ecto.obs['annotation_combined']


###Mesen
mesen.obs['annotation_combined'] = mesen.obs['annotation_2024']
mesen.obs['annotation_combined_fine'] = mesen.obs['E9_mesen_annotation_fine']
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_paper_coarse']

##add prefix
mesen.obs['annotation_combined'] = 'Mesenchyme - ' + mesen.obs['annotation_combined'].astype(str)
mesen.obs['annotation_combined'] = mesen.obs['annotation_combined'].astype('category')

mesen.obs['annotation_combined_fine'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_fine'].astype(str)
mesen.obs['annotation_combined_fine'] = mesen.obs['annotation_combined_fine'].astype('category')

mesen.obs['annotation_combined_coarse'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_coarse'].astype(str)
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_combined_coarse'].astype('category')

In [33]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [34]:
adata_combined.obs['annotation_combined_coarse'].value_counts()

annotation_combined_coarse
Mesenchyme - FNP Early Mesenchyme    366
Mesenchyme - MXP Early Mesenchyme     68
Ectoderm - Surface Ectoderm           56
Ectoderm - Ectodermal Placode         47
Ectoderm - Midfacial Ectoderm         23
Name: count, dtype: int64

In [35]:
adata_combined.obs['annotation_combined_fine'].value_counts()

annotation_combined_fine
Mesenchyme - Frontonasal Mesenchyme    227
Mesenchyme - Prrx2+ Gata3+ ?           139
Mesenchyme - Maxillary Mesenchyme       68
Ectoderm - Surface Ectoderm             56
Ectoderm - Ectodermal Placode           47
Ectoderm - Midfacial Ectoderm           23
Name: count, dtype: int64

In [36]:
adata_combined.write('anndata_objects/adata_E9_combined_annotated.h5ad')

In [37]:
#E10
ecto = sc.read('anndata_objects/adata_ecto_E10_nohesx1.h5ad')
#mesen = sc.read('anndata_objects/adata_E10_mesen_annotated.h5ad')
mesen = adata[adata.obs['stage']=='10'].copy()

In [38]:
mesen

AnnData object with n_obs × n_vars = 1527 × 19928
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'E8_mesen_annotation_coarse', 'E8_mesen_annotation_fine', 'integration_label', 'E9_mesen_annotation_coarse', 'E9_mesen_annotation_fine', 'E10_mesen_annotation_fine', 'E10_mesen_annotation_coarse', 'E11_mesen_annotation_coarse', 'E11_mesen_annotation_fine', 'E12_mesen_annotation_coarse', 'E13_mesen_annotation_coarse', 'E14_mesen_annotation_coarse', 'topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', '

In [39]:
###Ecto
ecto.obs['annotation_combined'] = ecto.obs['E10_ecto_annotation_coarse']

##add prefix
ecto.obs['annotation_combined'] = 'Ectoderm - ' + ecto.obs['annotation_combined'].astype(str)
ecto.obs['annotation_combined'] = ecto.obs['annotation_combined'].astype('category')
ecto.obs['annotation_combined_coarse'] = ecto.obs['annotation_combined']


###Mesen
mesen.obs['annotation_combined'] = mesen.obs['annotation_2024']
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_paper_coarse']


##add prefix
mesen.obs['annotation_combined'] = 'Mesenchyme - ' + mesen.obs['annotation_combined'].astype(str)
mesen.obs['annotation_combined'] = mesen.obs['annotation_combined'].astype('category')

mesen.obs['annotation_combined_coarse'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_coarse'].astype(str)
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_combined_coarse'].astype('category')

In [40]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [41]:
adata_combined

AnnData object with n_obs × n_vars = 2200 × 16923
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'annotation_combined', 'annotation_combined_coarse'
    obsm: 'X_MDE', 'X_pca', 'X_scvi', 'X_scvi_MDE', 'X_topic_compositions', 'X_umap', 'X_umap_features', '_scvi_extra_continuous_covs'
    layers: 'ambiguous', 'matrix', 'original_counts', 'spliced', 'unspliced'

In [42]:
adata_combined.obs['annotation_combined_coarse'].value_counts()

annotation_combined_coarse
Mesenchyme - MNP Mesenchyme                    738
Mesenchyme - MXP Early Mesenchyme              487
Mesenchyme - LNP Mesenchyme                    302
Ectoderm - Surface Ectoderm                    161
Ectoderm - Dorsomedial Olfactory Epithelium    159
Ectoderm - Ventral Olfactory Epithelium         75
Ectoderm - Olfactory Pit Rim - Six1             72
Ectoderm - Olfactory Pit Rim - Wnt6             66
Ectoderm - Dental Epithelium                    63
Ectoderm - Palate Epithelium                    52
Ectoderm - Committed Neuronal Progenitors       25
Name: count, dtype: int64

In [43]:
adata_combined.obs['annotation_combined'].value_counts()

annotation_combined
Mesenchyme - Dorsal NP                         388
Mesenchyme - Posterior MxP                     387
Mesenchyme - Medial NP                         350
Mesenchyme - Lateral NP                        302
Ectoderm - Surface Ectoderm                    161
Ectoderm - Dorsomedial Olfactory Epithelium    159
Mesenchyme - Anterior MxP                      100
Ectoderm - Ventral Olfactory Epithelium         75
Ectoderm - Olfactory Pit Rim - Six1             72
Ectoderm - Olfactory Pit Rim - Wnt6             66
Ectoderm - Dental Epithelium                    63
Ectoderm - Palate Epithelium                    52
Ectoderm - Committed Neuronal Progenitors       25
Name: count, dtype: int64

In [44]:
adata_combined.write('anndata_objects/adata_E10_combined_annotated.h5ad')

In [21]:
# adata_combined = adata_combined[adata_combined.obs['batch']=='adata10'].copy()
# adata_combined.write('anndata_objects/adata_E10_kaucka_combined_annotated.h5ad')

In [6]:
#E11
ecto = sc.read('anndata_objects/adata_ecto_E11_ofpr.h5ad')
mesen_old = sc.read('anndata_objects/adata_E11_mesen_annotation_2024.h5ad')

mesen = adata[adata.obs['stage']=='11'].copy()

In [5]:
ecto

AnnData object with n_obs × n_vars = 898 × 17204
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'leiden', 'leiden_ecto', 'E8_ecto_annotation_coarse', 'leiden_default', 'E9_ecto_annotation_coarse', 'reso_for_ecto', 'leiden_ecto_tmp', 'E10_ecto_annotation_coarse', 'leiden_ecto_fine', 'leiden_ecto_extra_fine', 'leiden_ecto1', 'E11_ecto_annotation_coarse', 'E11_ecto_annotation_fine', 'leiden_neuron', 'E12_ecto_annotation_coarse', 'E12_ecto_annotation_fine', 'leiden_coarse', 'E13_ecto_annotation_coarse', 'E13_ecto_annotation_fine', 'E14_ecto_annotation_coarse', 'E14_ecto_annotation_fine', 'annotation_tes

In [7]:
# mesen.obs['E11_mesen_annotation_main'] = mesen_old.obs['E11_mesen_annotation_main']
mesen.obs['E11_mesen_annotation_finest'] = mesen_old.obs['E11_mesen_annotation_finest']


In [8]:
###Ecto
ecto.obs['annotation_combined'] = ecto.obs['annotation_test']
ecto.obs['annotation_combined_fine'] = ecto.obs['E11_ecto_annotation_fine']

##add prefix
ecto.obs['annotation_combined'] = 'Ectoderm - ' + ecto.obs['annotation_combined'].astype(str)
ecto.obs['annotation_combined'] = ecto.obs['annotation_combined'].astype('category')

ecto.obs['annotation_combined_fine'] = 'Ectoderm - ' + ecto.obs['annotation_combined_fine'].astype(str)
ecto.obs['annotation_combined_fine'] = ecto.obs['annotation_combined_fine'].astype('category')
ecto.obs['annotation_combined_coarse'] = ecto.obs['annotation_combined']

###Mesen
mesen.obs['annotation_combined'] = mesen.obs['annotation_2024']
mesen.obs['annotation_combined_fine'] = mesen.obs['E11_mesen_annotation_finest']
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_paper_coarse']


##add prefix
mesen.obs['annotation_combined'] = 'Mesenchyme - ' + mesen.obs['annotation_combined'].astype(str)
mesen.obs['annotation_combined'] = mesen.obs['annotation_combined'].astype('category')

mesen.obs['annotation_combined_fine'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_fine'].astype(str)
mesen.obs['annotation_combined_fine'] = mesen.obs['annotation_combined_fine'].astype('category')

mesen.obs['annotation_combined_coarse'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_coarse'].astype(str)
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_combined_coarse'].astype('category')

In [9]:
ecto.obs['source']='ecto'
mesen.obs['source']='mesen'

In [10]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [11]:
adata_combined.obs['annotation_combined'].value_counts()

annotation_combined
Mesenchyme - Dorsal Nasal Cavity             624
Mesenchyme - LnP - Gata2                     550
Mesenchyme - lMxP                            540
Mesenchyme - pvMnP                           467
Mesenchyme - mMxP                            446
Mesenchyme - MxP - Lhx8                      407
Mesenchyme - mMnP                            337
Mesenchyme - Ventral Nasal Cavity            279
Mesenchyme - dMxP                            263
Mesenchyme - aMnP                            232
Mesenchyme - avMnP                           220
Ectoderm - Surface Ectoderm - Gjb6           158
Mesenchyme - LnP - Irx5                      144
Ectoderm - Ventral Olfactory Epithelium      129
Mesenchyme - LnP - Casz1                     112
Ectoderm - Vomeronasal Organ                  86
Ectoderm - Respiratory Epithelium             84
Ectoderm - Non-neuronal Progenitors           83
Ectoderm - Palate Epithelium                  68
Ectoderm - Surface Ectoderm - Dlx2            64


In [12]:
adata_combined.write('anndata_objects/adata_E11_combined_annotated.h5ad')

In [14]:
#E12
ecto = sc.read('anndata_objects/adata_ecto_E12_ofpr.h5ad')
mesen_old = sc.read('anndata_objects/mesen_E12_fine_annotation.h5ad')
mesen = adata[adata.obs['stage']=='12'].copy()

In [15]:
mesen.obs['E12_mesen_annotation_main'] = mesen_old.obs['E12_mesen_annotation_main']
mesen.obs['E12_mesen_annotation_finest'] = mesen_old.obs['E12_mesen_annotation_finest']


In [16]:
ecto.obs['E12_ecto_annotation_fine'].cat.categories

Index(['Committed Neuronal Progenitors', 'Conjunctival Epithelium',
       'Immature Neurons', 'Mature Neurons', 'Neuronal Progenitors',
       'Non-neuronal Progenitors', 'Olfactory Pit Rim - Fgf17',
       'Olfactory Pit Rim - Shh', 'Olfm5', 'Oral Epithelium',
       'Palate Epithelium', 'Periderm', 'Respiratory Epithelium',
       'Surface Ectoderm - Dlx2', 'Surface Ectoderm - Gjb6',
       'Surface Ectoderm - Gjb6/Rorb', 'Ventral Olfactory Epithelium',
       'Vomeronasal Organ', 'Whiskers - Ecto'],
      dtype='object')

In [17]:
mesen.obs['E12_mesen_annotation_finest'].cat.categories

Index(['Chondrocytes', 'Chondrocytes - Pax3', 'Chondrocytes - Tnc',
       'Dermal Condensate - Differentiated', 'Dermal Condensate - En1',
       'Dermal Condensate - Foxd1', 'FnP - Periocular', 'MxP - Aldh1a2',
       'MxP - Periocular', 'MxP Lip - Dlx1', 'MxP Lip - Fgf18',
       'MxP Lip - Irx1', 'MxP Lip - Tac1', 'Nasal Cavity - Anxa1',
       'Nasal Cavity - Anxa7', 'Nasal Cavity - Arg1', 'Nasal Cavity - Calca',
       'Nasal Cavity - Lox', 'Nasal Cavity - Nxph4', 'Nasal Cavity - Pou3f3',
       'Nasal Cavity - Tmem37', 'Nasal Cavity? - Irx3',
       'Nostril Fusion - Col13a1', 'Nostril Fusion - Mab21l2',
       'Nostrils - Casz1', 'Nostrils - Cpne4', 'Nostrils - Dab1',
       'Nostrils - Gata2', 'Nostrils - Pax7', 'Nostrils - Sim1',
       'Osteoblast - Dlx5', 'Osteomesenchyme - En1', 'Osteomesenchyme - Gata2',
       'Osteomesenchyme - MxP', 'Osteomesenchyme - Wnt16', 'Palate - Meox2',
       'Palate - Shox2', 'Philtrum', 'Primary Palate - Car2',
       'Primary Palate - Foxd1'

In [18]:
###Ecto
ecto.obs['annotation_combined'] = ecto.obs['annotation_test']
ecto.obs['annotation_combined_fine'] = ecto.obs['E12_ecto_annotation_fine']

##add prefix
ecto.obs['annotation_combined'] = 'Ectoderm - ' + ecto.obs['annotation_combined'].astype(str)
ecto.obs['annotation_combined'] = ecto.obs['annotation_combined'].astype('category')

ecto.obs['annotation_combined_fine'] = 'Ectoderm - ' + ecto.obs['annotation_combined_fine'].astype(str)
ecto.obs['annotation_combined_fine'] = ecto.obs['annotation_combined_fine'].astype('category')
ecto.obs['annotation_combined_coarse'] = ecto.obs['annotation_combined']

###Mesen
mesen.obs['annotation_combined'] = mesen.obs['annotation_2024']
mesen.obs['annotation_combined_fine'] = mesen.obs['E12_mesen_annotation_finest']
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_paper_coarse']

##add prefix
mesen.obs['annotation_combined'] = 'Mesenchyme - ' + mesen.obs['annotation_combined'].astype(str)
mesen.obs['annotation_combined'] = mesen.obs['annotation_combined'].astype('category')

mesen.obs['annotation_combined_fine'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_fine'].astype(str)
mesen.obs['annotation_combined_fine'] = mesen.obs['annotation_combined_fine'].astype('category')

mesen.obs['annotation_combined_coarse'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_coarse'].astype(str)
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_combined_coarse'].astype('category')

In [19]:
ecto.obs['source']='ecto'
mesen.obs['source']='mesen'

In [20]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [21]:
adata_combined.obs['annotation_combined_coarse'].value_counts()

annotation_combined_coarse
Mesenchyme - Whisker Pad                     1910
Mesenchyme - Nostrils                        1635
Mesenchyme - Nasal Cavity Mesenchyme         1556
Mesenchyme - Upper Lip                       1203
Mesenchyme - Palatal Shelf                    978
Mesenchyme - Osteomesenchyme                  922
Mesenchyme - Chondrocytes                     893
Mesenchyme - Maxillary Mesenchyme             550
Mesenchyme - Periocular Mesenchyme            536
Mesenchyme - VNO Mesenchyme                   401
Mesenchyme - Primary Palate                   308
Mesenchyme - Philtrum                         292
Mesenchyme - Osteoblasts                      267
Mesenchyme - Roof - Sim1                      243
Ectoderm - Surface Ectoderm - Dlx2            237
Ectoderm - Surface Ectoderm - Gjb6            194
Ectoderm - Ventral Olfactory Epithelium       143
Ectoderm - Olfactory Pit Rim - Six1           141
Ectoderm - Palate Epithelium                  112
Ectoderm - Non-neuronal

In [22]:
adata_combined.write('anndata_objects/adata_E12_combined_annotated.h5ad')

In [36]:
nasal_ecto = ['Committed Neuronal Progenitors','Immature Neurons', 'Mature Neurons', 'Neuronal Progenitors','Non-neuronal Progenitors',
 'Olfactory Pit Rim - Fgf17','Olfactory Pit Rim - Shh', 'Olfm5','Respiratory Epithelium','Ventral Olfactory Epithelium','Vomeronasal Organ',]
nasal_mesen = ['Nasal Cavity - Anxa1','Nasal Cavity - Anxa7', 'Nasal Cavity - Arg1', 'Nasal Cavity - Calca',
 'Nasal Cavity - Lox', 'Nasal Cavity - Nxph4', 'Nasal Cavity - Pou3f3','Nasal Cavity - Tmem37',
 'Nasal Cavity? - Irx3','VNO - Mesen',]

nasal_combined = ['Ectoderm - ' + s for s in nasal_ecto]+['Mesenchyme - ' + s for s in nasal_mesen]

In [37]:
adata_combined_nasal = adata_combined[adata_combined.obs['annotation_combined_fine'].isin(nasal_combined)].copy()

In [41]:
adata_combined_nasal.write('anndata_objects/adata_E12_combined_nasal.h5ad')

In [23]:
#E13
ecto = sc.read('anndata_objects/adata_ecto_E13_ofpr.h5ad')
mesen_old = sc.read('anndata_objects/adata_E13_mesen_filt_annotated.h5ad')
mesen = adata[adata.obs['stage']=='13'].copy()

In [24]:
mesen.obs['E13_mesen_fine_14_2024_annotation'] = mesen_old.obs['E13_mesen_fine_14_2024_annotation']
mesen.obs['E13_mesen_fine_14_2024'] = mesen_old.obs['E13_mesen_fine_14_2024']


In [25]:
###Ecto
ecto.obs['annotation_combined'] = ecto.obs['annotation_test']
ecto.obs['annotation_combined_fine'] = ecto.obs['E13_ecto_annotation_fine']

##add prefix
ecto.obs['annotation_combined'] = 'Ectoderm - ' + ecto.obs['annotation_combined'].astype(str)
ecto.obs['annotation_combined'] = ecto.obs['annotation_combined'].astype('category')

ecto.obs['annotation_combined_fine'] = 'Ectoderm - ' + ecto.obs['annotation_combined_fine'].astype(str)
ecto.obs['annotation_combined_fine'] = ecto.obs['annotation_combined_fine'].astype('category')
ecto.obs['annotation_combined_coarse'] = ecto.obs['annotation_combined']

###Mesen
mesen.obs['annotation_combined'] = mesen.obs['annotation_2024']
mesen.obs['annotation_combined_fine'] = mesen.obs['E13_mesen_fine_14_2024']
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_paper_coarse']

##add prefix
mesen.obs['annotation_combined'] = 'Mesenchyme - ' + mesen.obs['annotation_combined'].astype(str)
mesen.obs['annotation_combined'] = mesen.obs['annotation_combined'].astype('category')

mesen.obs['annotation_combined_fine'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_fine'].astype(str)
mesen.obs['annotation_combined_fine'] = mesen.obs['annotation_combined_fine'].astype('category')

mesen.obs['annotation_combined_coarse'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_coarse'].astype(str)
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_combined_coarse'].astype('category')

In [26]:
ecto.obs['source']='ecto'
mesen.obs['source']='mesen'

In [27]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [28]:
adata_combined.obs['annotation_combined'].value_counts()

annotation_combined
Mesenchyme - Chondrocytes - Septum/Capsule      1951
Mesenchyme - Nasal Cavity Mesenchyme            1753
Mesenchyme - Osteomesenchyme                    1546
Mesenchyme - Chondrocytes - Anterior Capsule     657
Mesenchyme - Lip - Dermal                        553
Mesenchyme - Roof - Sim1                         552
Mesenchyme - Palatal Shelf - Anterior            511
Mesenchyme - Lip Fibro - Dlk1                    503
Mesenchyme - Nostril leftovers                   410
Mesenchyme - Whisker Pad                         377
Mesenchyme - Primary Palate                      339
Mesenchyme - Palatal Shelf - Posterior           332
Mesenchyme - Philtrum                            323
Mesenchyme - MxP Mesenchyme                      319
Mesenchyme - VNO Mesenchyme                      314
Mesenchyme - Osteoblast                          296
Ectoderm - Non-neuronal Progenitors              222
Mesenchyme - Incisor                             206
Mesenchyme - Nasal Palate 

In [29]:
adata_combined.write('anndata_objects/adata_E13_combined_annotated.h5ad')

In [31]:
#E14
ecto = sc.read('anndata_objects/adata_ecto_E14_ofpr.h5ad')
mesen_old = sc.read('anndata_objects/adata_E14_mesen_annotation.h5ad')
mesen = adata[adata.obs['stage']=='14'].copy()

In [32]:
mesen.obs['E14_mesen_fine_14_2024'] = mesen_old.obs['E14_mesen_fine_14_2024']
mesen.obs['E14_mesen_annotation_14_2024'] = mesen_old.obs['E14_mesen_annotation_14_2024']


In [33]:
###Ecto
ecto.obs['annotation_combined'] = ecto.obs['annotation_test']
ecto.obs['annotation_combined_fine'] = ecto.obs['E14_ecto_annotation_fine']

##add prefix
ecto.obs['annotation_combined'] = 'Ectoderm - ' + ecto.obs['annotation_combined'].astype(str)
ecto.obs['annotation_combined'] = ecto.obs['annotation_combined'].astype('category')

ecto.obs['annotation_combined_fine'] = 'Ectoderm - ' + ecto.obs['annotation_combined_fine'].astype(str)
ecto.obs['annotation_combined_fine'] = ecto.obs['annotation_combined_fine'].astype('category')
ecto.obs['annotation_combined_coarse'] = ecto.obs['annotation_combined']

###Mesen
mesen.obs['annotation_combined'] = mesen.obs['annotation_2024']
mesen.obs['annotation_combined_fine'] = mesen.obs['E14_mesen_fine_14_2024']
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_paper_coarse']

##add prefix
mesen.obs['annotation_combined'] = 'Mesenchyme - ' + mesen.obs['annotation_combined'].astype(str)
mesen.obs['annotation_combined'] = mesen.obs['annotation_combined'].astype('category')

mesen.obs['annotation_combined_fine'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_fine'].astype(str)
mesen.obs['annotation_combined_fine'] = mesen.obs['annotation_combined_fine'].astype('category')
mesen.obs['annotation_combined_coarse'] = 'Mesenchyme - ' + mesen.obs['annotation_combined_coarse'].astype(str)
mesen.obs['annotation_combined_coarse'] = mesen.obs['annotation_combined_coarse'].astype('category')

In [34]:
ecto.obs['source']='ecto'
mesen.obs['source']='mesen'

In [35]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [36]:
adata_combined.obs['annotation_combined'].value_counts()

annotation_combined
Mesenchyme - Nasal Cavity Mesenchyme            422
Mesenchyme - Osteomesenchyme                    309
Mesenchyme - Chondrocytes - Septum/Capsule      181
Mesenchyme - Perichondrium                      159
Mesenchyme - Osteoblast                         147
Mesenchyme - Chondrocytes - Anterior Capsule    123
Mesenchyme - Periocular Mesenchyme              123
Mesenchyme - Philtrum                            84
Mesenchyme - Palatal Shelf - Posterior           77
Mesenchyme - MxP Mesenchyme                      74
Mesenchyme - Lip - Fibroblasts                   66
Mesenchyme - Nostril leftovers                   65
Ectoderm - Non-neuronal Progenitors              63
Mesenchyme - Roof - Sim1                         47
Mesenchyme - Dermal Condensate                   47
Mesenchyme - Palatal Shelf - Anterior            46
Ectoderm - Respiratory Epithelium                39
Mesenchyme - VNO Mesenchyme                      39
Mesenchyme - Primary Palate                 

In [37]:
adata_combined.write('anndata_objects/adata_E14_combined_annotated.h5ad')

In [9]:
#whisker cellchat attempts
#
#E11
ecto = sc.read('anndata_objects/adata_E11_surface.h5ad')
mesen = sc.read('anndata_objects/adata_E11_dermal.h5ad')

In [10]:
ecto

AnnData object with n_obs × n_vars = 333 × 22912
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'leiden', 'leiden_default', 'leiden_ecto', 'leiden_ecto_fine', 'leiden_ecto_extra_fine', 'leiden_ecto1', 'E11_ecto_annotation_coarse', 'E11_ecto_annotation_fine', 'leiden_surface', 'E11_ecto_surface_subset'
    var: 'gene_name', 'ensembl_id', 'mt', 'rb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'mean', 'std', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'E11_ecto_annotation_coarse_colors', 'E11_ecto_annotation_fin

In [11]:
mesen

AnnData object with n_obs × n_vars = 1455 × 22912
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'leiden', 'leiden_default', 'leiden_fine', 'E11_mesen_annotation_coarse', 'leiden_filt.8', 'leiden_filt.9', 'leiden_filt1.1', 'leiden_filt1.2', 'leiden_filt1.3', 'E11_mesen_annotation_fine', 'integration_label', 'leiden_dermal', 'E11_mesen_dermal_subset'
    var: 'gene_name', 'ensembl_id', 'mt', 'rb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'mean', 'std', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'E11_mesen_a

In [12]:
ecto.obs['annotation_combined'] = ecto.obs['E11_ecto_surface_subset']

mesen.obs['annotation_combined'] = mesen.obs['E11_mesen_dermal_subset']


In [13]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [14]:
adata_combined.write('anndata_objects/adata_E11_whiskers_combined.h5ad')

In [15]:
#whisker cellchat attempts
#
#E12
ecto = sc.read('anndata_objects/adata_E12_ecto_surface.h5ad')
mesen = sc.read('anndata_objects/adata_E12_mesen_dermal.h5ad')

In [16]:
ecto

AnnData object with n_obs × n_vars = 732 × 22912
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'leiden', 'leiden_default', 'leiden_ecto', 'leiden_neuron', 'E12_ecto_annotation_coarse', 'E12_ecto_annotation_fine', 'leiden_surface', 'leiden_surface_test', 'E11_ecto_surface'
    var: 'gene_name', 'ensembl_id', 'mt', 'rb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'mean', 'std', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'E11_ecto_surface_colors', 'E12_ecto_annotation_coarse_colors', 'E12_ecto_annotation_fine_

In [17]:
mesen

AnnData object with n_obs × n_vars = 3678 × 22912
    obs: 'Clusters', '_X', '_Y', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'sample', 'barcode', 'batch', 'doublet_score', 'predicted_doublets', 'predicted_doublets_based_on_10x_chromium_spec', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'pct_counts_unspliced', 'pred_doubl_cat', 'stage', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'leiden', 'leiden_lowres', 'leiden_default', 'leiden_filt2', 'leiden_filt.4', 'leiden_filt.5', 'leiden_filt1.5', 'E12_mesen_annotation_coarse', 'integration_label', 'leiden_dermal', 'leiden_dermal_high', 'E11_mesen_dermal'
    var: 'gene_name', 'ensembl_id', 'mt', 'rb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'mean', 'std', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'E11_mesen_dermal_colors', 'E12_mesen_annota

In [18]:
ecto.obs['annotation_combined'] = ecto.obs['E11_ecto_surface']

mesen.obs['annotation_combined'] = mesen.obs['E11_mesen_dermal']


In [19]:
import anndata as ad
adata_combined = ad.concat([ecto, mesen])

In [20]:
adata_combined.write('anndata_objects/adata_E12_whiskers_combined.h5ad')